In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

if os.environ["OPENAI_API_KEY"]:
    print('API Key is set')

API Key is set


In [2]:
from langchain_openai import ChatOpenAI 

In [3]:
llm=ChatOpenAI(model="gpt-5-nano", temperature=0)

## **RAG Implementation with PDFs**

#### **Step 1: Extracting text from PDFs**

In [4]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path='./Docs/fabric-admin.pdf'

loader=PyPDFLoader(pdf_path)

docs=loader.load()

docs

[Document(metadata={'producer': 'Microsoft Learn PDF 1.0.25309.01', 'creator': 'Microsoft Learn', 'creationdate': '2025-12-09T19:42:08+00:00', 'title': 'fabric admin | Microsoft Learn', 'moddate': '2025-12-09T19:42:08+00:00', 'source': './Docs/fabric-admin.pdf', 'total_pages': 290, 'page': 0, 'page_label': '1'}, page_content='Tell us about your PDF experience.\nMicrosoft Fabric documentation for\nadmins\nLearn about the Microsoft Fabric admin settings, options, and tools.\nFabric in your organization\nｅOVERVIEW\nWhat is Microsoft Fabric admin?\nWhat is the admin portal?\nｂGET STARTED\nEnable Fabric for your organization\nRegion availability\nFind your Fabric home region\nｃHOW-TO GUIDE\nUnderstand Fabric admin roles\nｉREFERENCE\nGovernance documentation\nSecurity documentation\nTools and settings\nｅOVERVIEW\nAbout tenant settings\nｃHOW-TO GUIDE\nSet up git integration\nSet up item certification'),
 Document(metadata={'producer': 'Microsoft Learn PDF 1.0.25309.01', 'creator': 'Microsoft 

### **Optional Creating own Metadata for PDF chunks**

In [5]:
for i in docs:
    i.metadata={'source':'fabric-admin.pdf',
                'developer':'Microsoft'}

In [6]:
docs[0].metadata

{'source': 'fabric-admin.pdf', 'developer': 'Microsoft'}

### **Step 2: Splitting the document into chunks**

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=10)

chunks=splitter.split_documents(docs)
chunks

[Document(metadata={'source': 'fabric-admin.pdf', 'developer': 'Microsoft'}, page_content='Tell us about your PDF experience.\nMicrosoft Fabric documentation for\nadmins\nLearn about the Microsoft Fabric admin settings, options, and tools.\nFabric in your organization\nｅOVERVIEW\nWhat is Microsoft Fabric admin?\nWhat is the admin portal?\nｂGET STARTED\nEnable Fabric for your organization\nRegion availability\nFind your Fabric home region\nｃHOW-TO GUIDE\nUnderstand Fabric admin roles\nｉREFERENCE\nGovernance documentation\nSecurity documentation\nTools and settings\nｅOVERVIEW\nAbout tenant settings\nｃHOW-TO GUIDE\nSet up git integration\nSet up item certification'),
 Document(metadata={'source': 'fabric-admin.pdf', 'developer': 'Microsoft'}, page_content='Configure notifications\nSet up metadata scanning\nEnable content certification\nEnable service principal authentication\nConfigure Multi-Geo support\nMonitoring and management\nｅOVERVIEW\nWhat is the admin monitoring workspace?\nｐCONCE

In [8]:
chunks[10].metadata

{'source': 'fabric-admin.pdf', 'developer': 'Microsoft'}

### **Step 3: Creating Embeddings fro the chunks**

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings_model=OpenAIEmbeddings(model="text-embedding-3-small")
# Without name also works
# embeddings_model=OpenAIEmbeddings()

### **Step 4: Store Embeddings in the Existing Local vector Store**

In [10]:
from langchain_community.vectorstores import Chroma

vectorstore=Chroma(
    persist_directory='./Vector/',
    embedding_function=embeddings_model
    )

C:\Users\Adity\AppData\Local\Temp\ipykernel_2456\3039456326.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore=Chroma(


In [12]:
vectorstore.add_documents(chunks)

['bd26ba83-244a-4235-8e51-d1dca1e743c7',
 '95ba1abf-a649-4bc5-9739-fe43206b37bb',
 '55d8bbca-ded6-4361-be13-6437aeb70809',
 '8b833e02-9235-424f-a06b-b58023662de4',
 'e60b0b22-2eb1-4033-a35a-0717d5fdcb18',
 'bee99a88-1115-4e4f-9356-3c2e78ee8e9d',
 '7d09b1d6-04ba-477f-bc4d-01550c2bb4f4',
 '11e9c6c9-67cf-4027-9e77-335b99b3fffd',
 '9f4c110c-2613-4c2b-ab37-c3d2560d8c71',
 '87d7c2ce-82f5-4b45-81f2-e15b324267b8',
 '305988d0-cce2-4148-ad10-3f90c2600305',
 '26703e0e-d984-49dd-b7a7-0635543304f8',
 '5d724aa9-53a9-405f-844f-1112ddfaa6c1',
 '6ca89e33-2dc5-4148-80c4-1aacaf066638',
 'c0de5ce6-3230-4524-b01a-37c7031c2b2b',
 '7874a4eb-398f-4371-9cfe-b1c269e7e5d0',
 '3bc9445a-cef1-4e1b-997a-351a8f99cf79',
 '12b544c0-8a12-4843-b67d-c1e63ee262f8',
 '22d47c00-214f-409e-afda-52e44a52a78b',
 '640ee141-2e50-47b3-aca5-727894f029c4',
 '45a72068-2461-41dc-8fc9-02b307d5cc31',
 '1ece5791-0677-4081-8010-106492a81134',
 '83085f58-5896-48b1-a0e3-ddbba0e43858',
 '3260ed26-cf9f-46c9-b225-efc22dc07832',
 '72370b76-adfc-

### **Step 5: Semantic Search**

In [ ]:
vectorstore.similarity_search('What is Microsoft Fabric admin?',k=3)


[Document(metadata={'source': 'fabric-admin.pdf', 'developer': 'Microsoft'}, page_content='What is Microsoft Fabric admin?\n07/01/2024\nMicrosoft Fabric admin is the management of the organization-wide settings that control how\nMicrosoft Fabric works. Users that are assigned to admin roles configure, monitor, and\nprovision organizational resources. This article provides an overview of admin roles, tasks, and\ntools to help you get started.\nThere are several roles that work together to administer Microsoft Fabric for your organization.\nMost admin roles are assigned in the Microsoft 365 admin portal or by using PowerShell. The\ncapacity admin roles are assigned when the capacity is created. To learn more about each of\nthe admin roles, see About admin roles. To learn how to assign admin roles, see Assign admin\nroles.\nThis section lists the Microsoft 365 admin roles and the tasks they can perform.\nGlobal administrator\nUnlimited access to all management features for the organizatio

### **Talk to LLM**

In [14]:
context=vectorstore.similarity_search('What is Microsoft Fabric admin?',k=3)
response=llm.invoke(f'What is Microsoft Fabric admin? you can answer based on the context: {context}')
print(response.content)

Microsoft Fabric admin refers to the set of organization-wide administration capabilities that control how Microsoft Fabric operates. It involves configuring, monitoring, and provisioning resources across the Fabric platform and governing access and usage.

Key points based on the provided material:
- Admins manage organization-wide settings, security, governance, and features for Fabric.
- Admin roles are used to perform these tasks; most roles are assigned in the Microsoft 365 admin portal or via PowerShell, with capacity/admin roles assigned when capacity is created.
- Two main admin roles for Fabric are Fabric administrator and Power Platform administrator. These roles grant access to the admin portal and control Fabric-wide settings and usage metrics, but do not include licensing rights.
- Licensing is handled separately (not part of Fabric admin roles).
- Admins can delegate responsibilities by creating groups of admins for different tasks (delegation helps avoid bottlenecks).
- 